***
__SimpleMask__ create mask from vlass values in a classification 
***

In [ ]:
import geemap
import ee
if not ee.data._credentials: ee.Initialize()

import sys
sys.path.append('../src/geepatches')
import geeutils
import geeproduct
import geemask

In [ ]:
eepoint       = geeutils.half31UESpoint #geeutils.bobspoint
eedatefrom    = geeutils.half31UESday   #geeutils.fleecycloudsday
eedatetill    = eedatefrom.advance(1, 'day')
roidiameter   = 64
eeroi         = geeutils.squareareaboundsroi(eepoint, roidiameter/2*20)
verbose       = False
#
# use experimental s2rgb product (using 'TCI' bands) as background
#
s2rgbimage    = geeproduct.GEECol_s2rgb().collect(eeroi, eedatefrom, eedatetill, verbose=verbose).first()

#
# making scl based mask
#
s2sclimage    = geeproduct.GEECol_s2scl().collect(eeroi, eedatefrom, eedatetill, verbose=verbose).first()

if False:
    #
    # make mask based on scene NOT being 2(dark), 4(vegetation), 5(not_vegetated), 6(water), 7(unclassified)
    #
    smmskimage    = geemask.SimpleMask([2, 4, 5, 6, 7], True).makemask(s2sclimage)
else:
    #
    # make mask based on scene being 0(?), 1(sat), 3(shadow), 8,9(cloud), 10(cirrus), 11(snow)
    #
    smmskimage    = geemask.SimpleMask([0, 1, 3, 8, 9, 10, 11]).makemask(s2sclimage)

print (geeutils.szprojectioninfo(smmskimage))
print (geeutils.szestimatevaluesinfo(smmskimage.clip(eeroi))) # clip; otherwise unbounded

if True:
    #
    # make clear sky transparant, clouds opaque
    #
    # SimpleMask:
    #    masked:     1 -> updateMask( img.eq(1)) == updateMask(True  (1)) -> img.mask() to 1 : gee clear  (opaque)
    #    clear :     0 -> updateMask( img.eq(1)) == updateMask(False (0)) -> img.mask() to 0 : gee masked (transparent)
    #    no data :   gee masked (default transparent)
    #
    smmskimage    = smmskimage.updateMask(smmskimage.eq(1))

    
map = geemap.Map(height='600px')
map.centerObject(eepoint, 13)
map.addLayer(s2rgbimage,                                       {'min':1,  'max':255},                  'rgb')
#map.addLayer(s2sclimage, geeutils.s2sclvisParams, 'scl')
map.addLayer(smmskimage,                                       {'min':0,  'max':1, 'opacity':0.8},     'msk')
map.addLayer(geeutils.outlinegeometryimage(eeroi, 0, 1),       {'palette':'#ff0000'})
map

In [ ]:
eepoint       = geeutils.half31UESpoint #geeutils.bobspoint
eedatefrom    = geeutils.hoogeheydeASCday #geeutils.half31UESday   #geeutils.fleecycloudsday
eedatetill    = eedatefrom.advance(1, 'day')
roidiameter   = 128
eeroi         = geeutils.squareareaboundsroi(eepoint, roidiameter/2*330)
verbose       = False
#
# use experimental pv333rgb product
#
pvrgbimage    = geeproduct.GEECol_pv333rgb().collect(eeroi, eedatefrom, eedatetill, verbose=verbose).first()
print (geeutils.szprojectioninfo(pvrgbimage))
print (geeutils.szestimatevaluesinfo(pvrgbimage.clip(eeroi))) # clip; otherwise unbounded

#
# making "pv status mask" based mask
#
pvsmvimage    = geeproduct.GEECol_pv333sm().collect(eeroi, eedatefrom, eedatetill, verbose=verbose).first()
#
# make mask based on scene NOT clear over sea or land
#
smmskimage    = geemask.SimpleMask([112, 120, 240, 248], True).makemask(pvsmvimage)
print (geeutils.szprojectioninfo(smmskimage))
print (geeutils.szestimatevaluesinfo(smmskimage.clip(eeroi))) # clip; otherwise unbounded

#
# clean: mask the rgb image with the mask
#
pvclnimage    =  pvrgbimage.updateMask(smmskimage.neq(1))
print (geeutils.szprojectioninfo(pvclnimage))
print (geeutils.szestimatevaluesinfo(pvclnimage.clip(eeroi))) # clip; otherwise unbounded

if True:
    #
    # make clear sky transparant, clouds opaque
    #
    # SimpleMask:
    #    masked:     1 -> updateMask( img.eq(1)) == updateMask(True  (1)) -> img.mask() to 1 : gee clear  (opaque)
    #    clear :     0 -> updateMask( img.eq(1)) == updateMask(False (0)) -> img.mask() to 0 : gee masked (transparent)
    #    no data :   gee masked (default transparent)
    #
    smmskimage    = smmskimage.updateMask(smmskimage.eq(1))

    
map = geemap.Map(height='600px')
map.centerObject(eepoint, 7)
map.addLayer(pvrgbimage,                                       {'min':0,  'max':1000},                 'rgb')
map.addLayer(smmskimage,                                       {'min':0,  'max':1, 'opacity':0.8},     'msk')
#map.addLayer(pvclnimage,                                       {'min':0,  'max':1000},                 'cln')
map.addLayer(geeutils.outlinegeometryimage(eeroi, 0, 5),       {'palette':'#ff0000'})
map